In [10]:
%matplotlib notebook
%cd /home/tessonec/devel/PySPG/test-notebook

%load_ext autoreload
%autoreload 2
import pandas as pd
import sys
sys.path.append("/mnt/hgfs/tessonec/Development/PySPG")

import spg.utils as spgu
from spg.plot.notebook import SPGDataLoader, BaseDataLoader, SPGInteractivePlotter

/mnt/hgfs/tessonec/Development/PySPG/test-notebook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
class CSVReader(BaseDataLoader):
    
    def __init__(self, ds_filename, output_columns = [], settings = {} ):
        self.full_dataframe = pd.read_csv(ds_filename)
        print "column_names:", [ k for k in self.full_dataframe.keys() ]
        
        self.output_columns = output_columns
        
        self.__initialise_independent_elements()
        print "non-constant columns:", self.variables

        self.independent_var =  self.variables[-1]
        
        try:
            self.coalesced_vars = [ self.variables[-2] ]
        except:
            self.coalesced_vars = []
        try:
            self.separated_vars = self.variables[:-2]
        except:
            self.separated_vars = []


        self.data = self.full_dataframe[self.variables + self.output_columns]
        
    def __initialise_independent_elements(self):        
        self.constants = {}
        self.variables = []
        for vn in self.full_dataframe.keys():
            if vn in self.output_columns:
                continue
            all_values = self.full_dataframe[vn].unique()
            if len(all_values) > 1: 
                self.variables.append( vn )
            else:
                self.constants[vn] = self.full_dataframe[vn].unique()[0]

    
    def configure_vars(self, separated_vars, coalesced_vars, independent_var, recalculate_output_columns = True):
        all_vars = separated_vars + coalesced_vars + [ independent_var ]
        
        # No unknown columns are present
        assert len( set( all_vars ) - set( self.data ) ) == 0
        # All sets are disjoint
        assert independent_var not in separated_vars
        assert independent_var not in coalesced_vars
        assert len( set(separated_vars).intersection( coalesced_vars ) ) ==  0
        
        self.separated_vars = separated_vars
        self.coalesced_vars = coalesced_vars
        self.independent_var = independent_var
        
        self.variables = all_vars
        
        if recalculate_output_columns:
            self.output_columns = [ kn for kn in self.data.keys()  if kn not in self.variables] 
        
        self.__initialise_independent_elements()
        
    
        
dip = CSVReader("psetting_t3_results.csv")

column_names: ['simulating_days', 'network_topology', 'num_nodes', 'pow_target_spacing', 'expo', 'nodes_conn', 'net_delay', 'max_branch', 'num_branch', 'orph_blc_rate', 'num_blocks', 'num_onchain_blocks', 'num_orph_blc', 'mean_prop_time', 'max_prop_time', 'repetition_time']
non-constant columns: ['num_nodes', 'nodes_conn', 'net_delay', 'max_branch', 'num_branch', 'orph_blc_rate', 'num_blocks', 'num_onchain_blocks', 'num_orph_blc', 'mean_prop_time', 'max_prop_time', 'repetition_time']


In [6]:
dip.configure_vars()

{1}